# P1963-computed: A graph recording the properties used in all instances of all classes

Wikidata has a property `properties for this type (P1963)` where editors can manually specify the properties that should be used to describe the instance of a class. 
This property is only defined for a small number of classes, and the properties listed are incomplete. In this use-case we compute the properties used for each class from the instances.

We invent a new property `P1963computed` to record the properties used in the instances of each class. In addition, we compute the counts, which we store as a quailifier `quantity (P1114)`

We compute two variants of these counts:
- `P1963computed` records the properties and counts for the direct instance of a class, includeing `P31x`
- `P1963computed_star` records the properties including the instances of every subclass below a class.

Properties computed in this notebook:
- `P31x`, a generalization of `instance of: P31`, sotred in `derived.P31x.tsv`
- `Pinstance_count`, counts the number of `P31/P31x` instances of a class, in `statistics.Pinstance_count.tsv.gz`
- `Pinstance_count_star`, same as `Pinstance_count`, but includes instances of the subclasses, in `statistics.Pinstance_count.star.tsv.gz`
- `P1963computed` in `derived.class.P1963computed.count.tsv.gz`
- `P1963computed_star` in `derived.P1963computed.count.star.tsv.gz`
- `Pproperty_domain` in `derived.Pproperty_domain.tsv.gz`

### `derived.P31x.tsv` contains the union of `P31` and `P31x`:
Below is an example of some of the edges for Arnold Schwarzenegger:

| node1 | label | node2 | id | node1;label | label;label | node2;label |
| -- | -- | -- | -- | -- | -- | -- |
| Q2685 | P31 | Q5 | Q2685-P31-Q5-eb1e33fe-0 | 'Arnold Schwarzenegger'@en | 'instance of'@en | 'human'@en |
| Q2685 | P31x | Q10800557 | Q2685-P31x-Q10800557 | 'Arnold Schwarzenegger'@en |  | 'film actor'@en |
| Q2685 | P31x | Q124100 | Q2685-P31x-Q124100 | 'Arnold Schwarzenegger'@en |  | 'bodybuilding'@en |
| Q2685 | P31x | Q131524 | Q2685-P31x-Q131524 | 'Arnold Schwarzenegger'@en |  | 'entrepreneur'@en |
| Q2685 | P31x | Q5 | Q2685-P31x-Q5 | 'Arnold Schwarzenegger'@en |  | 'human'@en |


### `Pinstance_count` and `Pinstance_count_star` examples

| node1 | label | node2 | id | node1;label |
| -- | -- | -- | -- | -- |
| Q1000017 | Pinstance_count | 1 | Q1000017-Pinstance_count-6b86b2 | 'Brentidae'@en |
| Q1000091 | Pinstance_count | 1 | Q1000091-Pinstance_count-6b86b2 | 'Rychnov nad Kněžnou'@en |
| Q1000156 | Pinstance_count | 11 | Q1000156-Pinstance_count-4fc82b |  |
| Q100023 | Pinstance_count | 1 | Q100023-Pinstance_count-6b86b2 | 'metasyntactic variable'@en |
| Q100026 | Pinstance_count | 1 | Q100026-Pinstance_count-6b86b2 | 'F-16'@en |

| node1 | label | node2 | id | node1;label |
| -- | -- | -- | -- | -- |
| Q1000068 | Pinstance_count_star | 13 | Q1000068-Pinstance_count_star-3fdba3 | 'Planungsverband'@en |
| Q100023 | Pinstance_count_star | 1 | Q100023-Pinstance_count_star-6b86b2 | 'metasyntactic variable'@en |
| Q100026 | Pinstance_count_star | 9 | Q100026-Pinstance_count_star-19581e | 'F-16'@en |
| Q100029091 | Pinstance_count_star | 10 | Q100029091-Pinstance_count_star-4a44dc |  |
| Q1000300 | Pinstance_count_star | 2 | Q1000300-Pinstance_count_star-d4735e | 'Land Rover Series'@en |

### How to use `derived.class.P1963computed.tsv.gz` and `derived.P1963computed.star.tsv.gz`

Here is an example query illustrating how to find the properties for a class, the counts and proportion:
```
kgtk("""
    query -i $OUT/derived.class.P1963computed.count.tsv.gz
        --match '
            (n1:Q100026)-[eid:P1963computed]->(n2), 
            (eid)-[:P1114]->(count),
            (eid)-[:P1107]->(proportion)'
        --return 'distinct n1 as node1, eid.label as label, n2 as node2, count as P1114, proportion as P1107'
        --order-by 'cast(P1114, int) desc'
        --limit 10
    / add-labels
    / md
""")
```
The results:

| node1 | label | node2 | P1114 | P1107 | node1;label | node2;label |
| -- | -- | -- | -- | -- | -- | -- |
| Q100026 | P1963computed | P17 | 1 | 0.015151515151515152 | 'F-16'@en | 'country'@en |
| Q100026 | P1963computed | P18 | 1 | 0.015151515151515152 | 'F-16'@en | 'image'@en |
| Q100026 | P1963computed | P31 | 1 | 0.015151515151515152 | 'F-16'@en | 'instance of'@en |
| Q100026 | P1963computed | P373 | 1 | 0.015151515151515152 | 'F-16'@en | 'Commons category'@en |
| Q100026 | P1963computed | P646 | 1 | 0.015151515151515152 | 'F-16'@en | 'Freebase ID'@en |


Here is the equivalent query on the `derived.P1963computed.star.tsv.gz`
```
kgtk("""
    query -i $OUT/derived.P1963computed.count.star.tsv.gz
        --match '
            (n1:Q100026)-[eid:P1963computed_star]->(n2), 
            (eid)-[:P1114]->(count),
            (eid)-[:P1107]->(proportion)'
        --return 'distinct n1 as node1, eid.label as label, n2 as node2, count as P1114, proportion as P1107'
        --order-by 'cast(P1114, int) desc'
        --limit 10
    / add-labels
    / md
""")
```
Note that there are many more results for the `P1963computed_star` property as it includes the instances of the subclasses of F-16. For example, there are 10 instances of subclassess of F-16 that have a `P31` property, and that is about 15% of all instances of subclasses of F-16, so many don't have a `P31`, which is add:

| node1 | label | node2 | P1114 | P1107 | node1;label | node2;label |
| -- | -- | -- | -- | -- | -- | -- |
| Q100026 | P1963computed_star | P31 | 10 | 0.15151515151515152 | 'F-16'@en | 'instance of'@en |
| Q100026 | P1963computed_star | P361 | 8 | 0.12121212121212122 | 'F-16'@en | 'part of'@en |
| Q100026 | P1963computed_star | P729 | 8 | 0.12121212121212122 | 'F-16'@en | 'service entry'@en |
| Q100026 | P1963computed_star | P426 | 7 | 0.10606060606060606 | 'F-16'@en | 'aircraft registration'@en |
| Q100026 | P1963computed_star | P176 | 6 | 0.09090909090909091 | 'F-16'@en | 'manufacturer'@en |
| Q100026 | P1963computed_star | P730 | 6 | 0.09090909090909091 | 'F-16'@en | 'service retirement'@en |
| Q100026 | P1963computed_star | P137 | 4 | 0.06060606060606061 | 'F-16'@en | 'operator'@en |
| Q100026 | P1963computed_star | P373 | 2 | 0.030303030303030304 | 'F-16'@en | 'Commons category'@en |
| Q100026 | P1963computed_star | P17 | 1 | 0.015151515151515152 | 'F-16'@en | 'country'@en |
| Q100026 | P1963computed_star | P18 | 1 | 0.015151515151515152 | 'F-16'@en | 'image'@en |

### Example of `Pproperty_domain`

Here is a query to get all the domains of `cast member (P161)`
```
kgtk("""
    query -i $OUT/derived.Pproperty_domain.tsv.gz --idx mode:graph
        --match '
            (property:P161)-[edge_id:Pproperty_domain]->(class),
            (edge_id)-[:P1114]->(instance_count),
            (edge_id)-[:P1107]->(proportion)'
        --return 'distinct property as node1, "Pproperty_domain" as label, class as node2, instance_count as P1114, proportion as P1107'
        --order-by 'property, cast(P1114, float)*cast(P1107, float)  desc'
    / add-labels
""")
```

For example, there are 171111 instances of subclasses of film that have cast member, and that is about 54% of all instances of subclasses of film:

| node1 | label | node2 | P1114 | P1107 | node1;label | node2;label |
| -- | -- | -- | -- | -- | -- | -- |
| P161 | Pproperty_domain | Q11424 | 171111 | 0.5425945832818044 | 'cast member'@en | 'film'@en |
| P161 | Pproperty_domain | Q10301427 | 197378 | 0.38757120500167885 | 'cast member'@en | 'moving image'@en |
| P161 | Pproperty_domain | Q79782 | 197379 | 0.3420157510331742 | 'cast member'@en | 'motion'@en |
| P161 | Pproperty_domain | Q17320256 | 197379 | 0.34154997819661803 | 'cast member'@en | 'physical process'@en |
| P161 | Pproperty_domain | Q2431196 | 198160 | 0.3382660960616991 | 'cast member'@en | 'audiovisual work'@en |
| P161 | Pproperty_domain | Q758901 | 198160 | 0.3382649412013731 | 'cast member'@en | 'audiovisual media'@en |
| P161 | Pproperty_domain | Q58898430 | 198160 | 0.33826436377416724 | 'cast member'@en | 'electronic supplementary materials'@en |
| P161 | Pproperty_domain | Q58901077 | 198160 | 0.33826320892566947 | 'cast member'@en | 'publication format'@en |

### Preamble: set up the environment and files used in the tutorial

In [1]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [2]:
# Parameters

kgtk_path = "/Users/pedroszekely/Documents/GitHub/kgtk"

# Folder on local machine where to create the output and temporary folders
# input_path = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/"
# input_path = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215-dwd-v2/"
# input_path = "/Volumes/saggu-ssd/wikidata-dwd-v3"
input_path="/data/amandeep/wikidata-20220505-dwd-v4"
# output_path = "/Users/pedroszekely/Downloads/kypher/projects"
# output_path = "/Volumes/saggu-ssd/wikidata-dwd-v3"
output_path = "/data/amandeep/wikidata-20220505-dwd-v4"

graph_cache_path = "/Volumes/crucial/wikidata-cache/wikidata.sqlite3.db"

project_name = "p1963"
debug = True
files = "claims,item,datatypes,p279,p279star"
files_for_cache = None

In [3]:
files = files.split(',')

if files_for_cache is None:
    files_for_cache =  files
else:
    files_for_cache = files_for_cache.split(",")

Our Wikidata distribution partitions the knowledge in Wikidata into smaller files that make it possible for you to pick and choose which files you want to use:

In [4]:
ck = ConfigureKGTK(files, kgtk_path=kgtk_path)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  graph_cache_path=graph_cache_path,
                  debug=True)

User home: /nas/home/amandeep
Current dir: /data/amandeep/Github/kgtk-notebooks/use-cases/create_wikidata
KGTK dir: /Users/pedroszekely/Documents/GitHub/kgtk
Use-cases dir: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases


The KGTK setup command defines environment variables for all the files so that you can reuse the Jupyter notebook when you install it on your local machine.

In [5]:
ck.print_env_variables()

KGTK_GRAPH_CACHE: /data/amandeep/wikidata-20220505-dwd-v4/p1963/temp.p1963/wikidata.sqlite3.db
STORE: /data/amandeep/wikidata-20220505-dwd-v4/p1963/temp.p1963/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
kgtk: kgtk --debug
KGTK_LABEL_FILE: /data/amandeep/wikidata-20220505-dwd-v4/labels.en.tsv.gz
kypher: kgtk --debug query --graph-cache /data/amandeep/wikidata-20220505-dwd-v4/p1963/temp.p1963/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/examples
TEMP: /data/amandeep/wikidata-20220505-dwd-v4/p1963/temp.p1963
OUT: /data/amandeep/wikidata-20220505-dwd-v4/p1963
GRAPH: /data/amandeep/wikidata-20220505-dwd-v4
USE_CASES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases
claims: /data/amandeep/wikidata-20220505-dwd-v4/claims.tsv.gz
item: /data/amandeep/wikidata-20220505-dwd-v4/claims.wikibase-item.tsv.gz
datatypes: /data/amandeep/wikidata-20220505-dwd-v4/metadata.property.datatypes.tsv.gz
p279: /data/amandeep/wikidata-20220505-dwd-v4/derived.P279.tsv.gz
p279

In [ ]:
ck.load_files_into_cache(files=files_for_cache)

Specify the indices we want for our files as the queries we do in this notebook are very expensive:
- `claims:` indexed as a graph because we query based on node1, label and node2
- `p279star:` indexed as a monograph because the file has a single property.

In [ ]:
!kgtk --debug query -i claims --idx mode:graph --limit 5

In [ ]:
!kgtk --debug query -i item --idx mode:graph --limit 5

In [ ]:
# Example of how to clear the indices and reindex
# !kgtk --debug query -i claims --idx mode:clear mode:graph -i p279star --idx mode:clear mode:monograph --limit 5

## Define `P31x`, a generalization of `instance of: P31`

In our KG we are going to define a new property called `instance of (generalized): P31x` that behaves like DBpedia, so that we can ask for instances of `artist: Q483501`.
We do this by generalizing `occupation: P106` abd `position held: 39` to also behave as `P31` statements.

Approach:
- Combine `x P31 y`, `x P106 y` and `x P39 y` statements using a new `P31x` predicate

In [ ]:
kgtk("""
    query -i claims
        --match '
            (n1)-[:P31]->(:Q5),
            (n1)-[r {label: property}]->(n2)'
        --where 'property in ["P106", "P39"]'
        --return 'distinct n1 as node1, "P31xQ5" as label, n2 as node2'
    / add-id --id-style wikidata
    / cat -i - -i "$p31"
    / compact
    -o $OUT/derived.P31x.tsv.gz
""")

Load the `p31x` graph defining our generalized `instance of` property:

In [ ]:
kgtk("""
        query -i $OUT/derived.P31x.tsv.gz --as p31x --limit 2
    """)

## Instance Counts

In [6]:
kgtk("""
    query -i p31x
        --match '(instance)-[]->(class)'
        --return 'distinct class as node1, "Pinstance_count" as label, count(distinct instance) as node2'
        --where 'instance != class'
        --order-by 'class'
    / add-id --id-style wikidata
    -o $OUT/statistics.Pinstance_count.tsv.gz
""")

In [ ]:
if debug:
    kgtk("head -i $OUT/statistics.Pinstance_count.tsv.gz -n 5 / add-labels")

In [ ]:
!kgtk --debug query -i $OUT/statistics.Pinstance_count.tsv.gz --as p31xcount --idx mode:graph --limit 3

In [7]:
kgtk("""
    query -i p31x -i p279star
        --match '
            p31x: (instance)-[]->(hop),
            p279star: (hop)-[]->(class)'
        --where 'instance != hop'
        --return 'distinct class as node1, "Pinstance_count_star" as label, count(distinct instance) as node2'
    -o $OUT/statistics.Pinstance_count.star.tsv.gz
""")

In [ ]:
!kgtk --debug query -i $OUT/statistics.Pinstance_count.star.tsv.gz --as p31xcountstar --idx mode:graph --limit 3

In [ ]:
if debug:
    kgtk("head -i $OUT/statistics.Pinstance_count.star.tsv.gz -n 5 / add-labels")

### Added by Amandeep, to get the properties for this class for subclasses


The following query will count the number of subclasses for every super class

In [8]:
kgtk(""" --debug
    query -i p279star
        --match 'p279star: (class)-[]->(super_class)'
        --where 'class != super_class'
        --return 'distinct super_class as node1, "Psubclass_count_star" as label, count(distinct class) as node2'
    -o $OUT/statistics.Psubclass_count.star.tsv.gz
""")

[2022-06-13 15:36:28 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_5_c1."node2" "_aLias.node1", ? "_aLias.label", count(DISTINCT graph_5_c1."node1") "_aLias.node2"
     FROM graph_5 AS graph_5_c1
     WHERE (graph_5_c1."node1" != graph_5_c1."node2")
     GROUP BY "_aLias.node1", "_aLias.label"
  PARAS: ['Psubclass_count_star']
---------------------------------------------



In [ ]:
!kgtk --debug query -i $OUT/statistics.Psubclass_count.star.tsv.gz --as subclasscountstar --idx mode:graph --limit 3

In [ ]:
kgtk(""" --debug
    query -i subclasscountstar1
        --match '(:Q62049)-[]->()'
""")

In [ ]:
if debug:
    kgtk("head -i $OUT/statistics.Psubclass_count.star.tsv.gz -n 5 / add-labels")

## Compute the number of times each property appears in a class

In this section we will compute the distribution of the use of properties in every class in th KG. 
We want to know the count of the different properties used in all instance of a class.
For example, if we look at `film (Q11424)` we want to see what properties are used to describe films, including all subclasses of film.

Computing this distribution is challenging because we want to include the instances of subclasses, and class hierarchy in Wikidata is ver large.

Approach:
- Compute a graph that represents the properties used to describe an entity. We will use `Phas_property` to record the properties used in each entity.
- Aggregate the counts over all the instances of a class, inlcuding instances of the subclasses.
  - Exact method: forl each class, enumerate all the subclasses, and for each subclass enumerate all its intances, and then count the ocurrences of the properties. The exact method works for small graphs, but for DWD (Wikidata scale), the exact method will not return a result even after 2 days.
  - Approximate method: 
    - Count the distribution of properties for all instances of a class, not including subclasses
    - For every class, sum the property counts over the direct children of the class. This is a recursive definition, as we need to calculate the sum for the children before that we can aggregate up to the paprent. KGTK does not support recursive queries, so we will do this in a dumb way, so for every class we add the number of all the subclasses and below
    
  > The parent/child implementation of the approximate method is approximate because we may overcount classes: given classes `A` and `B` such that `A` is below `B`, if there are multiple distinct paths from `A` to `B`, we will double count. The advantage of the dumb method is that it does not double-count

### Compute the `Phas_property`, the set of properties used in each entity

In [ ]:
# use for subject properties
kypher = "claims: (entity)-[l {label: property}]->()"
# use for object properties
# kypher = "claims: ()-[l {label: property}]->(entity)"

kgtk(f"""
    query -i p31x -i claims
        --match '
            p31x: (entity)-[]->(),
            {kypher}'
        --return 'distinct entity as node1, "Phas_property" as label, property as node2'
    / add-id --id-style wikidata
    -o $TEMP/item_properties.tsv.gz
""")



### Get the properties for this class including subclasses

The following cell gets the properties for subclass instead of instances of subclasses

In [ ]:
# use for subject properties
kypher = "claims: (entity)-[l {label: property}]->()"
# use for object properties
# kypher = "claims: ()-[l {label: property}]->(entity)"

kgtk(f"""
    query -i p279star -i claims
        --match '
            p279star: (entity)-[]->(),
            {kypher}'
        --return 'distinct entity as node1, "Phas_property" as label, property as node2'
    / add-id --id-style wikidata
    -o $TEMP/item_properties_subclasses.tsv.gz
""")



### Compute `P1963computed`

For every class record the count of each property in all instances of the class according to the `P31x` graph which contains the `P31x` and `P31` statements.

In [9]:
kgtk("""
    query -i p31x -i $TEMP/item_properties.tsv.gz
        --match '  
            p31x: (entity)-[]->(class),
            item_properties: (entity)-[l]->(property)'
        --where 'entity != class'
        --return 'distinct class as node1, "P1963computed" as label, property as node2, count(distinct l) as P1114'
    / add-id --id-style wikidata
    / normalize --add-id True
    -o $OUT/derived.class.P1963computed.tsv.gz
""")

### Added by Amandeep, to get the properties for this class for subclasses
For every class, record the count of each property in all subclasses of the class

In [10]:
kgtk("""
    query -i p279star -i $TEMP/item_properties_subclasses.tsv.gz
        --match '  
            p279star: (entity)-[]->(class),
            item_properties_subclasses: (entity)-[l]->(property)'
        --where 'entity != class'
        --return 'distinct class as node1, "P1963subclass_star" as label, property as node2, count(distinct l) as P1114'
    / add-id --id-style wikidata
    / normalize --add-id True
    -o $OUT/derived.class.P1963computed.subclass.tsv.gz
""")

Calculate the proportion of instances of a class that have each property, recording the proportion using property `proportion (P1107)`

In [11]:
kgtk("""
    query -i $OUT/derived.class.P1963computed.tsv.gz -i p31xcount
        --match '
            computed:
                (class)-[edge_id:P1963computed]->(property),
                (edge_id)-[:P1114]->(property_count),
            count:
                (class)-[]->(instance_count)'
        --return 'edge_id as node1, "P1107" as label, cast(property_count, float)/cast(instance_count, float) as node2'
    / add-id --id-style wikidata
    / cat -i - -i $OUT/derived.class.P1963computed.tsv.gz
    / sort
    -o $OUT/derived.class.P1963computed.count.tsv.gz
""")

In the second step, we use `P279star` to iterate over all classes, finding all subclasses below each superclass, and we sum the counts for the properties.


In the return clause, we list `superclass`, and the value of the `property` variable ahead of the `count` clause to tell KGTK that we want to aggregate by superclass and property. We reuse the Wikidata `quantity (P1114)` to record the counts:

In [12]:
kgtk("""
    query -i $OUT/derived.class.P1963computed.tsv.gz -i p279star \
    --match ' 
        P1963computed: 
            (class)-[edge_id]->(property), 
            (edge_id)-[:P1114]->(count),
        p279star: 
            (class)-[:P279star]->(superclass)' 
    --return 'distinct superclass as node1, "P1963computed_star" as label, property as node2, sum(count) as P1114'
    / add-id --id-style wikidata
    / normalize --add-id True
    -o "$OUT"/derived.P1963computed.star.tsv.gz
""")
#

The following query claculates `P1963computed_x` in one go, by counting over all instances of all sublasses of each superclass. This query is too expensive: it will not complete after 2 days when running on DWD.

Calculate the proportion of instances of a class that have each property, recording the proportion using property `proportion (P1107)`

In [13]:
kgtk("""
    query -i $OUT/derived.P1963computed.star.tsv.gz -i p31xcountstar
        --match '
            computed:
                (class)-[edge_id:P1963computed_star]->(property),
                (edge_id)-[:P1114]->(property_count),
            count:
                (class)-[]->(instance_count)'
        --return 'edge_id as node1, "P1107" as label, cast(property_count, float)/cast(instance_count, float) as node2'
    / add-id --id-style wikidata
    / cat -i - -i $OUT/derived.P1963computed.star.tsv.gz
    / sort
    -o $OUT/derived.P1963computed.count.star.tsv.gz
""")

### Added by Amandeep, to get the properties for this class for subclasses
version 2, subclasses only

In [14]:
kgtk("""
    query -i "$OUT"/derived.class.P1963computed.subclass.tsv.gz -i subclasscountstar
        --match '
            computed:
                (class)-[edge_id:P1963subclass_star]->(property),
                (edge_id)-[:P1114]->(property_count),
            count:
                (class)-[]->(instance_count)'
        --return 'edge_id as node1, "P1107" as label, cast(property_count, float)/cast(instance_count, float) as node2'
    / add-id --id-style wikidata
    / cat -i - -i "$OUT"/derived.class.P1963computed.subclass.tsv.gz
    / sort
    -o $OUT/derived.P1963computed.subclass.count.star.tsv.gz
""")

## Compute `Pproperty_domain` that maps properties to the classes whose instances use the property

In [ ]:
kgtk("""
    query -i $OUT/derived.P1963computed.count.star.tsv.gz --idx mode:graph
        --match '
            (class)-[edge_id:P1963computed_star]->(property),
            (edge_id)-[:P1114]->(instance_count),
            (edge_id)-[:P1107]->(proportion)'
        --return 'distinct property as node1, "Pproperty_domain" as label, class as node2, instance_count as P1114, proportion as P1107'
        --order-by 'property, cast(P1114, int) desc'
    / add-id --id-style wikidata
    / normalize --add-id True
    -o $OUT/derived.Pproperty_domain.tsv.gz
""")